In [ ]:
%%python module Camunda

from js import XMLHttpRequest
from urllib.parse import urlencode
from IPython.display import display

import json
import os

def get(path, params=None):
    request = XMLHttpRequest.new()
    request.open("GET", f"{os.environ['CAMUNDA_ENGINE_API']}{path}?{urlencode(params or {})}", False)
    request.send(None)
    assert request.status in [200, 204], request.responseText
    return json.loads(request.responseText or "null")

def post(path, data=None):
    request = XMLHttpRequest.new()
    request.setHeader("X-XSRF-TOKEN", os.environ["CAMUNDA_CSRF_TOKEN"])
    request.setHeader("Content-Type", "application/json")
    request.open("POST", f"{os.environ['CAMUNDA_ENGINE_API']}{path}", False)
    request.send(json.dumps(data))
    assert request.status in [200, 204], request.responseText
    return json.loads(request.responseText or "null")

class Camunda:

    def __init__(self):
        self.process_id = None
    
    def get_definitions(self):
        results = get("/process-definition", dict(latest=True))
        return [d["processDefinitionKey"] for d in results]
    
    def start_instance(self, key):
        instance = post(f"/process-definition/key/{key}/start")

In [ ]:
*** Settings ***

Library  Camunda

In [ ]:
*** Tasks ***

WIP
    Get deployments